<a href="https://colab.research.google.com/github/MitchMathieu/cisc452-a3/blob/master/CISC452_a3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CISC 452 - Assignment 3
IMPLEMENT A HYBRID COMPETITIVE LEARNING NETWORK

In [1]:
! git clone https://github.com/MitchMathieu/cisc452-a3.git
! ls cisc452-a3

fatal: destination path 'cisc452-a3' already exists and is not an empty directory.
iris_test.txt  iris_train.txt


In [0]:
import numpy as np
import random

In [0]:
def read_txt(file_path):
  x = []
  y = []
  f = open(file_path, 'r')
  for line in f:
    line = line.rstrip().split(',')
    x.append(np.array(line[:-1], dtype=np.float32))
    y.append(line[-1])
  return np.array(x), np.array(y)

In [0]:
train_x, train_y = read_txt("cisc452-a3/iris_train.txt")
test_x, test_y = read_txt("cisc452-a3/iris_test.txt")
# print(train_x)
# print(train_y)

Encoding

In [0]:
def cat_to_vector(label):
  # one hot encode the flower type
  if label == "Iris-setosa":
    return np.array([1, 0, 0], dtype=np.int)
  elif label == "Iris-versicolor":
    return np.array([0, 1, 0], dtype=np.int)
  elif label == "Iris-virginica":
    return np.array([0, 0, 1], dtype=np.int)
    
def vector_to_cat(vec):
  if np.array_equal(vec, [1, 0, 0]):
    return "Iris-setosa"
  elif np.array_equal(vec, [0, 1, 0]):
    return "Iris-versicolor"
  elif np.array_equal(vec, [0, 0, 1]):
    return "Iris-virginica"
  
def onehot_iris(arr):
  arr_enc = []
  for cat in arr:
    vec = cat_to_vector(cat)
    arr_enc.append(vec)
  return np.array(arr_enc)

In [0]:
train_y_enc = onehot_iris(train_y)
test_y_enc = onehot_iris(test_y)
# print(test_y_enc)

# LVQ1
LVQ algorithm
* init random weights
* assign each output node to a class 
* find output node whose weight vector is closest to input vec
* move weight vec closer if correct, further if incorrect
* repeat until convergence

In [0]:
from math import sqrt

In [0]:
class LVQ:
  def __init__(self, num_inputs, num_outputs):
    self.weights = np.random.rand(num_outputs, num_inputs)
#     self.weights = [self.random_codebook(train_x) for i in range(num_outputs)]
  
#   def random_codebook(self, train):
#     n_records = len(train)
#     n_features = len(train[0])
#     codebook = [train[random.randrange(n_records)][i] for i in range(n_features)]
#     return codebook
    
  def weight_to_class(self, cv):
    arr = np.zeros(3)
    for i, w in enumerate(self.weights):
      if np.array_equal(w, cv):
        arr[i] = 1
    return arr
    
  def euclidian_dist(self, vec1, vec2):
    distance = 0.0
    for i in range(len(vec1)):
      distance += (vec1[i] - vec2[i])**2
    return sqrt(distance)
  
  def find_closest_vec(self, pattern):
    distances = []
    for vec in self.weights:
      dist = self.euclidian_dist(pattern, vec)
      distances.append((vec, dist))
    distances.sort(key=lambda tup: tup[1])
#     print(f"dist: {distances[0][1]}")
    return distances[0][0]
  
  def train(self, train_x, train_y, lrate, epochs):
    for epoch in range(epochs):
      rate = lrate * (1.0 - (epoch/float(epochs)))
      sum_err = 0.0
      for pattern, true_label in zip(train_x, train_y):
        closest_vec = self.find_closest_vec(pattern)
#         print(f"pattern:{pattern}")
#         print(f"closest_vec:{closest_vec}")
        pred_label = self.weight_to_class(closest_vec)
#         print(f"pred_label:{pred_label}, true_label:{true_label}")
        for i in range(pattern.size):
          error = pattern[i] - closest_vec[i]
          sum_err += error**2
          if np.array_equal(pred_label, true_label):
            closest_vec[i] += rate * error
          else:
            closest_vec[i] -= rate * error
#         print(f"closest_vec adjusted:{closest_vec}\n")
      print(f">epoch={epoch+1}, lrate={rate:.3f}, error={sum_err:.3f}")
      
  def test(self, test_x, test_y):
    correct = 0.0
    for pattern, label in zip(test_x, test_y):
      pred = self.weight_to_class(self.find_closest_vec(pattern))
      if np.array_equal(pred, label):
        correct += 1
      print(f"actual={label} got={pred}")
    print(f"correct={correct}")
    print(f"final accuracy={(correct/len(test_y)):.4f}")

In [26]:
lvq1 = LVQ(4, 3)
print(f"init weights:\n{lvq1.weights}")

init weights:
[[0.27200805 0.16147137 0.49609398 0.87873938]
 [0.57438324 0.05070407 0.07582354 0.25360191]
 [0.01313549 0.82082695 0.14912425 0.79979378]]


In [0]:
#Shuffles x, y and onehot array entirely! Very quick!
def shuffle(x,y,onehot):
    indices = [i for i in range(len(x))]
    np.random.shuffle(indices)
    return x[indices],y[indices],onehot[indices]

In [28]:
train_x, train_y, train_y_enc = shuffle(train_x,
                                       train_y,train_y_enc)

lvq1.train(train_x, train_y_enc, 0.1, 10)
print(f"trained weights:\n{lvq1.weights}")

>epoch=1, lrate=0.100, error=4635.252
>epoch=2, lrate=0.090, error=905.649
>epoch=3, lrate=0.080, error=738.924
>epoch=4, lrate=0.070, error=779.662
>epoch=5, lrate=0.060, error=794.082
>epoch=6, lrate=0.050, error=809.176
>epoch=7, lrate=0.040, error=825.782
>epoch=8, lrate=0.030, error=843.393
>epoch=9, lrate=0.020, error=856.168
>epoch=10, lrate=0.010, error=865.119
trained weights:
[[ 4.8638871   3.85716064  0.43160484 -0.15806962]
 [-1.20023164 -1.57864636 -0.17061819  0.29831402]
 [ 7.18929583  3.07178136  7.40887362  3.29093846]]


In [29]:
lvq1.test(test_x, test_y_enc)

actual=[1 0 0] got=[1. 0. 0.]
actual=[1 0 0] got=[1. 0. 0.]
actual=[1 0 0] got=[1. 0. 0.]
actual=[1 0 0] got=[1. 0. 0.]
actual=[1 0 0] got=[1. 0. 0.]
actual=[1 0 0] got=[1. 0. 0.]
actual=[1 0 0] got=[1. 0. 0.]
actual=[1 0 0] got=[1. 0. 0.]
actual=[1 0 0] got=[1. 0. 0.]
actual=[1 0 0] got=[1. 0. 0.]
actual=[0 1 0] got=[1. 0. 0.]
actual=[0 1 0] got=[0. 0. 1.]
actual=[0 1 0] got=[1. 0. 0.]
actual=[0 1 0] got=[0. 0. 1.]
actual=[0 1 0] got=[1. 0. 0.]
actual=[0 1 0] got=[0. 0. 1.]
actual=[0 1 0] got=[0. 0. 1.]
actual=[0 1 0] got=[1. 0. 0.]
actual=[0 1 0] got=[0. 0. 1.]
actual=[0 1 0] got=[1. 0. 0.]
actual=[0 0 1] got=[0. 0. 1.]
actual=[0 0 1] got=[0. 0. 1.]
actual=[0 0 1] got=[0. 0. 1.]
actual=[0 0 1] got=[0. 0. 1.]
actual=[0 0 1] got=[0. 0. 1.]
actual=[0 0 1] got=[0. 0. 1.]
actual=[0 0 1] got=[0. 0. 1.]
actual=[0 0 1] got=[0. 0. 1.]
actual=[0 0 1] got=[0. 0. 1.]
actual=[0 0 1] got=[0. 0. 1.]
correct=20.0
final accuracy=0.6667


# PCA - ANN Approach


In [0]:
class PCA:
  def __init__(self, num_inputs, num_outputs):
    self.weights = np.random.rand(num_outputs, num_inputs)
    
  def update_weights(self, pattern):
    pass
  
  def get_output(self, pattern):
    arr = []
    np.dot()
  
  def train(self, train_x, lrate, iterations):
    for iteration in range(iterations):
      print(f"\nIteration={iteration+1}")
      for pattern in train_x:
        print(f"pattern=pattern")
        print(f"weights={self.weights}")
        for vec in self.weights:
          dp = np.dot(vec, pattern)
          print(f"vec={vec}, dp={dp}")
          vec += (lrate * dp * pattern) - (dp**2) * vec
          print(f"updated vec={vec}")
        print(f"\n")
#       print(f"weights:\n{self.weights}")
    

In [40]:
pca = PCA(4, 3)
print(pca.weights)

[[0.73167183 0.40829273 0.69764981 0.64937079]
 [0.54556126 0.44125527 0.79669302 0.74955098]
 [0.48519923 0.24654837 0.15705781 0.28880013]]


In [41]:
pca.train(train_x, 0.1, 1)



Iteration=1
pattern=pattern
weights=[[0.73167183 0.40829273 0.69764981 0.64937079]
 [0.54556126 0.44125527 0.79669302 0.74955098]
 [0.48519923 0.24654837 0.15705781 0.28880013]]
vec=[0.73167183 0.40829273 0.69764981 0.64937079], dp=5.2034190835477165
updated vec=[-16.84128972  -9.0854399  -17.61924042 -16.88067946]
vec=[0.54556126 0.44125527 0.79669302 0.74955098], dp=4.6209967664729295
updated vec=[ -9.11711303  -7.59483905 -15.70727034 -15.20985915]
vec=[0.48519923 0.24654837 0.15705781 0.28880013], dp=3.0276454900111966
updated vec=[-2.66055855 -1.10517751 -0.94959314 -2.32824946]


pattern=pattern
weights=[[-16.84128972  -9.0854399  -17.61924042 -16.88067946]
 [ -9.11711303  -7.59483905 -15.70727034 -15.20985915]
 [ -2.66055855  -1.10517751  -0.94959314  -2.32824946]]
vec=[-16.84128972  -9.0854399  -17.61924042 -16.88067946], dp=-228.47138924132753
updated vec=[878942.94738091 474177.13134668 919593.9643409  881110.97112641]
vec=[ -9.11711303  -7.59483905 -15.70727034 -15.20985915

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: overflow encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in subtract


updated vec=[nan nan nan nan]


pattern=pattern
weights=[[nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]]
vec=[nan nan nan nan], dp=nan
updated vec=[nan nan nan nan]
vec=[nan nan nan nan], dp=nan
updated vec=[nan nan nan nan]
vec=[nan nan nan nan], dp=nan
updated vec=[nan nan nan nan]


pattern=pattern
weights=[[nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]]
vec=[nan nan nan nan], dp=nan
updated vec=[nan nan nan nan]
vec=[nan nan nan nan], dp=nan
updated vec=[nan nan nan nan]
vec=[nan nan nan nan], dp=nan
updated vec=[nan nan nan nan]


pattern=pattern
weights=[[nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]]
vec=[nan nan nan nan], dp=nan
updated vec=[nan nan nan nan]
vec=[nan nan nan nan], dp=nan
updated vec=[nan nan nan nan]
vec=[nan nan nan nan], dp=nan
updated vec=[nan nan nan nan]


pattern=pattern
weights=[[nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]]
vec=[nan nan nan nan], dp=nan
updated vec=[nan nan nan nan]
vec=[nan nan nan nan], dp=nan
upda